In [ ]:
import numpy as np

# # setup
---

In [ ]:
%cd /data/kimgh/CenterPoint-static/det3d/ops/iou3d_nms

!python setup.py build_ext --inplace

# # Prepare data
---

In [ ]:
%cd /data/kimgh/CenterPoint

!python create_data.py nuscenes_data_prep --root_path '/data/kimgh/CenterPoint/data/nuScenes/' --version 'v1.0-mini' --nsweeps 10

In [16]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-dynamic

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-dynamic/data'
sensor ='lidar'

!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor}

/data/kimgh/CenterPoint-custom/CenterPoint-dynamic
no apex
2023-04-11 13:37:56.986535: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-11 13:37:58.113162: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
No Tensorflow
['A/source/A_Clip_07689_03/Lidar/2-048_07689_LR_064.pcd', 'A/source/A_Clip_07689_03/Lidar/2-048_07689_LR_057.pcd', 'A/source/A_Clip_07689_03/Lidar/2-048_07689_LR_012.pcd', 'A/source/A_Clip_07689_03/Lidar/2-048_07689_LR_078.pcd', 'A/source/A_Clip_07689_03/Lidar/2-048_07689_LR_049.pcd', 'A/source/A_Clip_07689_03/Lidar/2-048_07689_LR_001.pcd', 'A/source/A_Clip_07689_03/Lidar/2-048_07689_LR_042.pcd', 'A/source/A_Clip_07689_03/Lidar/2-048_07689_LR_100.pcd', 'A/source/A_Clip_07689_03/Lidar/2-

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-static/data'
sensor = 'lidar'

!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor}

# # Train
---

In [ ]:
%cd /data/kimgh/CenterPoint

CONFIG_PATH = '/data/kimgh/CenterPoint/configs/nusc/voxelnet/nusc_centerpoint_voxelnet_01voxel.py'
work_dir = './result/nusc_centerpoint_voxelnet_01voxel'

f'!python -m torch.distributed.launch --nproc_per_node 4 train.py {CONFIG_PATH} --work_dir {work_dir} --validate'

In [1]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

CONFIG_PATH = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_normal.py'
work_dir = './result/lidar/train'

!python train.py --config {CONFIG_PATH} --work_dir {work_dir} --validate

/data/kimgh/CenterPoint-custom/CenterPoint-static
no apex
2023-04-12 10:19:58.616135: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-12 10:20:00.131938: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
No Tensorflow
Deformable Convolution not built!
No APEX!
2023-04-12 10:20:02,418 - INFO - Sensor: lidar
2023-04-12 10:20:02,418 - INFO - Config: /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_normal.py
2023-04-12 10:20:02,418 - INFO - Distributed training: False
2023-04-12 10:20:02,418 - INFO - torch.backends.cudnn.benchmark: False
2023-04-12 10:20:02,563 - INFO - Finish RPN Initialization
2023-04-12 10:20:02,564 - INFO - num_classes: [1, 1, 

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

CONFIG_PATH = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_normal_copy.py'
work_dir = './result/lidar/train'

f'!python -m torch.distributed.launch --nproc_per_node 4 train.py --config {CONFIG_PATH} --work_dir {work_dir} --validate'

In [23]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-dynamic

CONFIG_PATH = '/data/kimgh/CenterPoint-custom/CenterPoint-dynamic/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel.py'
work_dir = './result/lidar/train'

f'!python -m torch.distributed.launch --nproc_per_node 4 train.py --config {CONFIG_PATH} --work_dir {work_dir} --validate'

/data/kimgh/CenterPoint-custom/CenterPoint-dynamic


'!torchrun --nproc_per_node 4 train.py --config /data/kimgh/CenterPoint-custom/CenterPoint-dynamic/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel.py --work_dir ./result/lidar/train --validate'

# # Validation
---

In [ ]:
# lidar-normal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_normal.py'
work_dir = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/lidar/normal_allsubid'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/weights/nia_centerpoint_static_lidar.pth'
sensor = 'lidar'

f'!CUDA_VISIBLE_DEVICES=1 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test' 

In [ ]:
# lidar-abnormal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_abnormal.py'
work_dir = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/lidar/abnormal_allsubid'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/weights/nia_centerpoint_static_lidar.pth'
sensor = 'lidar'

f'!CUDA_VISIBLE_DEVICES=2 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'

In [ ]:
# radar-normal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar.py'
work_dir = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/radar/normal'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/weights/nia_centerpoint_static_radar.pth'
sensor = 'radar'

!CUDA_VISIBLE_DEVICES=0 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test

In [ ]:
# radar-abnormal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_abnormal.py'
work_dir = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/radar/abnormal'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/weights/nia_centerpoint_static_radar.pth'

!CUDA_VISIBLE_DEVICES=3 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --speed_test